<a href="https://colab.research.google.com/github/audio-bible/scripts/blob/master/Youtube_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Youtube Downloader

Download audio from [the KJV Audio Bible Channel on youtube](https://www.youtube.com/@skiesaboveus/playlists) of all bible chapters and save to google drive (My Drive/kjv_audio) sorted by chapters. Futher processing may be needed to make use of the audio.

You will likely exceed  google drive quota. So we suggest that once the download is done, you download the directory and do futher processing elsewhere



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
!pip install --upgrade audio-bible-tools


In [46]:
from audio_bible_tools import YouTubeDownloader
root_dir = '/content/drive/My Drive/'
project_dir = 'kjv_audio'
channel_handle = 'skiesaboveus'
genesis_playlist = "01 Genesis Chapters 01-50 (KJV)"
revelation_playlist = "66 Revelation Chapters 01-22 (KJV)"

# Initialize the downloader
downloader = YouTubeDownloader(
    root_dir=root_dir,
    channel_handle=channel_handle,
    project_dir=project_dir,
    genesis_playlist=genesis_playlist,
    revelation_playlist=revelation_playlist
)

# Remove any temporary files
downloader.remove_tmp_files()

# Print the total number of files in the project directory
# print(downloader.count_files_in_directory())

downloader.list_and_process_playlists()

# Remove temporary files again after download
downloader.remove_tmp_files()

# Process the downloaded files
downloader.version("kjv") \
    .voice("kjv-audio-bible") \
    .result_dir("bible_audio_processed") \
    .chapterPattern("Chapter (\d+)") \
    .process()

Processing Playlist Title: 40 Matthew Chapters 01-28 (KJV)


 28 of 28 expected chapters
Skipping Playlist Title: 40 Matthew Chapters 01-28 (KJV) (Already has 28 chapters)
----------------------------------------
Processing Playlist Title: 41 Mark Chapters 01-16 (KJV)


 16 of 16 expected chapters
Skipping Playlist Title: 41 Mark Chapters 01-16 (KJV) (Already has 16 chapters)
----------------------------------------
Processing Playlist Title: 42 Luke Chapters 01-24 (KJV)


 24 of 24 expected chapters
Skipping Playlist Title: 42 Luke Chapters 01-24 (KJV) (Already has 24 chapters)
----------------------------------------
Processing Playlist Title: 43 John Chapters 01-21 (KJV)


 21 of 21 expected chapters
Skipping Playlist Title: 43 John Chapters 01-21 (KJV) (Already has 21 chapters)
----------------------------------------
Processing Playlist Title: 44 Acts Chapters 01-28 (KJV)


...

 27 of 27 expected chapters
Skipping Playlist Title: 03 Leviticus Chapters 01-27 (KJV) (Already has 27 chapters)
----------------------------------------
Processing Playlist Title: 02 Exodus Chapters 01-40 (KJV)


 40 of 40 expected chapters
Skipping Playlist Title: 02 Exodus Chapters 01-40 (KJV) (Already has 40 chapters)
----------------------------------------
Processing Playlist Title: 01 Genesis Chapters 01-50 (KJV)


 50 of 50 expected chapters
Skipping Playlist Title: 01 Genesis Chapters 01-50 (KJV) (Already has 50 chapters)
----------------------------------------
Processing Playlist Title: 27 Daniel Chapters 01-12 (KJV)


 12 of 12 expected chapters
Skipping Playlist Title: 27 Daniel Chapters 01-12 (KJV) (Already has 12 chapters)
----------------------------------------
Processing Playlist Title: 29 Joel Chapters 01-03 (KJV)


 3 of 3 expected chapters
Skipping Playlist Title: 29 Joel Chapters 01-03 (KJV) (Already has 3 chapters)
----------------------------------------
Ignored Playlist Title: Old Testament
Playlist URL: https://www.youtube.com/playlist?list=PLZNQlwBptV_HWNvJTh2DzV4nIXnFWrzwB
----------------------------------------
Ignored Playlist Title: New Testament
Playlist URL: https://www.youtube.com/playlist?list=PLZNQlwBptV_Filw-P8QtXWTe9BCblD0IO
----------------------------------------
Ignored Playlist Title: Videos
Playlist URL: https://www.youtube.com/playlist?list=PLZNQlwBptV_EqjLKRn_cMMrN-RLzCx49u
----------------------------------------
Skipping existing file: /content/drive/My Drive/bible_audio_processed/Joel/001/kjv/kjv-audio-bible.m4a from /content/drive/My Drive/kjv_audio/29_Joel_Chapters_01-03/The Holy Bible - Joel Chapter 1 (KJV).m4a
Skipping existing file: /content/drive/My Drive/bible_audio_processed/Joel/002/kjv/kjv-audio-bible.m4a from /content/drive/My Drive/kjv_audio/29_Joel_Chapte